In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.font_manager import FontProperties
from pyecharts import options as opts
from pyecharts.charts import Bar, Line, Boxplot,Timeline,Map
from pyecharts.globals import ThemeType
import matplotlib.font_manager as fm
import warnings
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams["font.family"] = ["SimHei"]
plt.rcParams['axes.unicode_minus'] = False
rc = {'font.sans-serif': 'SimHei','axes.unicode_minus': False}
sns.set(context='notebook', style='ticks', rc=rc, font_scale=1.2)

In [21]:
!!! # 去掉这行再运行就行。因为一整个文件运行完了有40mb，太占内存了。

["'!' 不是内部或外部命令，也不是可运行的程序", '或批处理文件。']

In [ ]:
df_ab = pd.read_csv('IS_availability.csv')
df_cd = pd.read_csv('TH_country_day.csv')
df_cld = pd.read_csv('TH_country_lang_day.csv')
df_nnu = pd.read_csv('TH_num_new_user.csv')

df_list = [df_ab, df_cd, df_cld, df_nnu]
for df in df_list:
    print("数据基本信息：")
    print(df.info())
    print("\n缺失值统计：")
    print(df.isnull().sum())
    df.dropna(inplace=True)
    print(df.isnull().sum())

### （一）分析点1：国家层面图像使用频率（img_perc）分析

In [ ]:
# 提取所需字段并检查数据
df_ab_filtered = df_ab[['country', 'date', 'img_perc']].copy()
print("数据基本信息：")
df_ab_filtered.info()

# 检查缺失值
missing_values = df_ab_filtered.isnull().sum()
print("\n缺失值统计：")
# print(missing_values)

In [ ]:
# 按国家分组计算img_perc的均值和标准差
country_img_stats = df_ab_filtered.groupby('country')['img_perc'].agg(['mean', 'std']).reset_index()
country_img_stats.columns = ['country', 'img_perc_mean', 'img_perc_std']

# 筛选前20个国家
top_20_countries = country_img_stats.sort_values('img_perc_mean', ascending=False).head(20)
top_50_countries = country_img_stats.sort_values('img_perc_mean', ascending=False).head(50)

plt.figure(figsize=(16, 10))
bars = plt.bar(top_20_countries['country'], top_20_countries['img_perc_mean'], 
               color=sns.color_palette("viridis", 20), zorder=3)

plt.title('全球图像使用频率前20国家', fontsize=20)
plt.xlabel('国家代码（ISO 3166-1）', fontsize=14)
plt.ylabel('图像密度均值（img_perc）', fontsize=14)
plt.xticks( ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

# 为前5和后15国家添加不同标签样式
for i, bar in enumerate(bars):
    height = bar.get_height()
    if i < 5:  # 前5国家用较大字体标注
        plt.text(bar.get_x() + bar.get_width()/2, height + 0.01,
                 f'{height:.4f}', ha='center', va='bottom', fontsize=14, weight='bold')
    else:
        plt.text(bar.get_x() + bar.get_width()/2, height + 0.01,
                 f'{height:.4f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('top20_img_perc.png', dpi=300)
plt.show()

print(top_50_countries['country'])

In [ ]:
# 定义前10发展中国家和前10发达国家列表（基于你提供的国家列表筛选）
# 注意：以下列表需根据实际数据中前50国家的排序结果调整，此处仅为示例
developing_top10 = [
    'CN', 'PN', 'VI', 'MM', 'EH',  # 前5个发展中国家
    'MO', 'BV', 'VN', 'NR', 'NC'   # 后5个发展中国家
]
developed_top10 = [
    'US', 'JP', 'UK', 'DE', 'FR',  # 前5个发达国家
    'CA', 'AU', 'IT', 'NL', 'SE'   # 后5个发达国家
]

# 筛选前10发展中国家和前10发达国家的数据
developing_data = country_img_stats[country_img_stats['country'].isin(developing_top10)]
developed_data = country_img_stats[country_img_stats['country'].isin(developed_top10)]

# 合并数据
comparison_data = pd.concat([developing_data, developed_data])
comparison_data['country_type'] = comparison_data['country'].apply(
    lambda x: '发展中国家' if x in developing_top10 else '发达国家'
)

# 1. 绘制分组柱状图对比前10发展中国家和前10发达国家
plt.figure(figsize=(16, 10))
sns.barplot(x='country', y='img_perc_mean', hue='country_type', data=comparison_data, zorder=3)

plt.title('前10发达国家与前10发展中国家图像使用频率对比', fontsize=20)
plt.xlabel('国家代码', fontsize=14)
plt.ylabel('图像密度均值(img_perc)', fontsize=14)
plt.xticks(ha='right')
plt.legend(title='国家类型', fontsize=12, loc='best')

# 增强图表可读性
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

# 为每个柱子添加数值标签
for i, bar in enumerate(plt.gca().patches):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.005,
             f'{height:.4f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('top10_developed_vs_developing.png', dpi=300)
plt.show()

# 2. 绘制箱线图与小提琴图组合，展示分布特征
plt.figure(figsize=(14, 8))
sns.violinplot(x='country_type', y='img_perc_mean', data=comparison_data, 
               palette=['#00A1FF', '#5ed935'], zorder=3)
sns.boxplot(x='country_type', y='img_perc_mean', data=comparison_data, 
            color='white', width=0.1, zorder=4)

plt.title('前10发达国家与前10发展中国家图像使用频率分布对比', fontsize=18)
plt.xlabel('国家类型', fontsize=14)
plt.ylabel('图像密度均值(img_perc)', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

plt.tight_layout()
plt.savefig('top10_img_perc_distribution.png', dpi=300)
plt.show()

# 3. 计算并显示统计差异（均值、中位数、标准差）
developing_stats = developing_data['img_perc_mean'].describe()
developed_stats = developed_data['img_perc_mean'].describe()

# 输出详细统计量
print("\n前10发展中国家图像密度统计量：")
print(developing_stats)
print("\n前10发达国家图像密度统计量：")
print(developed_stats)

# 计算均值差异与中位数差异
mean_diff = developed_stats['mean'] - developing_stats['mean']
median_diff = developed_stats['50%'] - developing_stats['50%']

print(f"\n发达国家与发展中国家均值差异：{mean_diff:.6f}")
print(f"发达国家与发展中国家中位数差异：{median_diff:.6f}")

# 4. 绘制抖动图展示个体差异
plt.figure(figsize=(12, 7))
sns.stripplot(x='country_type', y='img_perc_mean', data=comparison_data, 
              jitter=True, alpha=0.6, color='gray', zorder=2)
sns.pointplot(x='country_type', y='img_perc_mean', data=comparison_data, 
              color='black', markers='o', linestyles='-', zorder=3)

plt.title('前10发达国家与前10发展中国家图像使用频率个体分布', fontsize=16)
plt.xlabel('国家类型', fontsize=14)
plt.ylabel('图像密度均值(img_perc)', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

plt.tight_layout()
plt.savefig('top10_img_perc_jitter.png', dpi=300)
plt.show()

# 5. 绘制前10国家的img_perc_std对比图
plt.figure(figsize=(16, 10))
sns.barplot(x='country', y='img_perc_std', hue='country_type', data=comparison_data, zorder=3)

plt.title('前10发达国家与前10发展中国家图像使用频率波动性对比', fontsize=18)
plt.xlabel('国家代码', fontsize=14)
plt.ylabel('图像密度标准差(img_perc_std)', fontsize=14)
plt.xticks(rotation=60, ha='right')
plt.legend(title='国家类型', fontsize=12, loc='best')

# 增强图表可读性
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

# 为每个柱子添加数值标签
for i, bar in enumerate(plt.gca().patches):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.005,
             f'{height:.4f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('top10_img_perc_std_comparison.png', dpi=300)
plt.show()

In [ ]:
# 改进后的大洲映射数据（基于ISO 3166-1标准）
continent_map = {
    'US': '北美洲', 'JP': '亚洲', 'IN': '亚洲', 'BR': '南美洲',
    'UK': '欧洲', 'DE': '欧洲', 'FR': '欧洲', 'CN': '亚洲',
    'RU': '欧洲', 'MX': '北美洲', 'CA': '北美洲', 'AU': '大洋洲',
    'IT': '欧洲', 'NL': '欧洲', 'SE': '欧洲', 'ID': '亚洲', 'ZA': '非洲',
    'TR': '亚洲', 'AR': '南美洲', 'PE': '南美洲', 'KZ': '亚洲', 'MM': '亚洲',
    'VN': '亚洲', 'BD': '亚洲', 'NG': '非洲', 'EG': '非洲', 'BR': '南美洲',
    'AR': '南美洲', 'AU': '大洋洲', 'NZ': '大洋洲', 'ZA': '非洲', 'SD': '非洲'
}

# 转换为DataFrame
continent_df = pd.DataFrame(list(continent_map.items()), columns=['country', 'continent'])

# 合并大洲数据与国家图像统计数据
continent_stats = country_img_stats.merge(continent_df, on='country', how='inner')

# 绘制小提琴图与箱线图组合
plt.figure(figsize=(14, 8))
sns.violinplot(x='continent', y='img_perc_mean', data=continent_stats, 
               palette=sns.color_palette("pastel", len(continent_stats['continent'].unique())), zorder=3)
sns.boxplot(x='continent', y='img_perc_mean', data=continent_stats, 
            color='white', width=0.1, zorder=4)

plt.title('各大洲图像使用频率分布对比', fontsize=18)
plt.xlabel('大洲', fontsize=14)
plt.ylabel('图像密度均值（img_perc）', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

# 调整x轴标签旋转角度
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.savefig('continent_img_perc_distribution.png', dpi=300)
plt.show()

In [ ]:
df_ab_filtered['date'] = pd.to_datetime(df_ab_filtered['date'])

# 选择对比国家（美国、日本、中国、印度）
comparison_countries = ['US', 'JP', 'CN', 'IN']
country_names = {'US': '美国', 'JP': '日本', 'CN': '中国', 'IN': '印度'}

# 筛选数据并重置索引
multi_country_data = df_ab_filtered[df_ab_filtered['country'].isin(comparison_countries)]
multi_country_data = multi_country_data.reset_index(drop=True)
multi_country_data['date'] = pd.to_datetime(multi_country_data['date'])

# 分步计算90日移动平均（解决索引匹配问题）
def calculate_rolling_mean(group):
    return group.rolling(window=90).mean()

# 使用apply分组计算并重置索引
rolling_means = multi_country_data.groupby('country')['img_perc'].apply(calculate_rolling_mean)
rolling_means = rolling_means.reset_index(drop=True)  # 重置索引以匹配原数据框

# 将计算结果添加为新列
multi_country_data['img_perc_ma'] = rolling_means
multi_country_data = multi_country_data.dropna()
multi_country_data = multi_country_data.reset_index(drop=True)

# ------------------- 多国趋势对比图 -------------------
colors = ['#00A1FF', '#5ed935', '#f8ba00', '#ff2501']
plt.figure(figsize=(18, 10))

for i, country in enumerate(comparison_countries):
    country_data = multi_country_data[multi_country_data['country'] == country]
    if not country_data.empty:
        # 计算相对于初始值的百分比变化
        initial_value = country_data['img_perc_ma'].iloc[0]
        country_data['relative_change'] = (country_data['img_perc_ma'] / initial_value - 1) * 100
        
        # 绘制相对变化趋势线
        plt.plot(country_data['date'], country_data['relative_change'], 
                 label=f"{country_names[country]} (基准={initial_value:.2f})", 
                 color=colors[i], linewidth=2, zorder=3)
        
        # 标记峰值（基于相对变化）
        peak_idx = country_data['relative_change'].idxmax()
        peak_date = country_data.loc[peak_idx, 'date']
        peak_value = country_data.loc[peak_idx, 'relative_change']
        plt.scatter(peak_date, peak_value, color=colors[i], s=100, zorder=4)
        plt.annotate(f"{country_names[country]}峰值: {peak_value:.1f}%", (peak_date, peak_value),
                    xytext=(10, 10), textcoords='offset points',
                    arrowprops=dict(arrowstyle='->'), fontsize=12)

# 设置图表样式
plt.title('各国图像密度相对变化趋势对比（相对于初始值的百分比）', fontsize=20)
plt.xlabel('日期', fontsize=14)
plt.ylabel('相对变化百分比 (%)', fontsize=14)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))
plt.xticks(rotation=45, ha='right')
plt.legend(loc='best', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('relative_change_trend.png', dpi=300)
plt.show()

# ------------------- 季节性特征分析（多国对比版 · 适配你的数据） -------------------
# 修正1：仅用原始对比国家（US、JP、CN、IN），或补充GB数据到df_ab_filtered
# 如果你要加英国，需确保df_ab_filtered包含'GB'数据，否则会空图
comparison_countries_season = ['US', 'JP', 'CN', 'IN']  # 保持与原始数据一致
country_names_season = {'US':'美国', 'JP':'日本', 'CN':'中国', 'IN':'印度'}
country_colors_season = ['#00A1FF', '#5ed935', '#f8ba00', '#ff2501']  # 复用颜色

# 定义季节区间（通用北半球）
summer_months = [6,7,8]   # 夏季：6-8月
winter_months = [12,1,2]  # 冬季：12-2月（跨年度）

# 创建画布（2行2列，匹配4个国家）
plt.figure(figsize=(16, 10))

for i, country in enumerate(comparison_countries_season):
    # 筛选国家数据（用你的multi_country_data）
    country_data = multi_country_data[multi_country_data['country'] == country]
    if country_data.empty:
        plt.subplot(2, 2, i+1).axis('off')
        continue
    
    # 提取月份并计算月度均值
    country_data['month'] = country_data['date'].dt.month
    monthly_mean = country_data.groupby('month')['img_perc_ma'].mean()
    
    # 绘制子图
    ax = plt.subplot(2, 2, i+1)
    sns.barplot(
        x=monthly_mean.index, 
        y=monthly_mean.values, 
        color=country_colors_season[i], 
        zorder=3,
        ax=ax
    )
    
    # 标记季节区间（核心：用 month-1 转x轴索引）
    # 夏季区间（6-8月 → 索引5-7）
    for month in summer_months:
        ax.axvspan(
            month - 1 - 0.4, 
            month - 1 + 0.4, 
            color='lightblue', 
            alpha=0.3, 
            zorder=1
        )
    # 冬季区间（12、1、2月 → 索引11、0、1）
    for month in winter_months:
        ax.axvspan(
            month - 1 - 0.4, 
            month - 1 + 0.4,  
            color='lightgreen', 
            alpha=0.3, 
            zorder=1
        )
    
    # 设置子图样式
    ax.set_title(f"{country_names_season[country]} 季节性分布", fontsize=14)
    ax.set_xlabel('月份', fontsize=12)
    ax.set_ylabel('图像密度均值', fontsize=12)
    ax.set_xticklabels(
        ['1月','2月','3月','4月','5月','6月', 
         '7月','8月','9月','10月','11月','12月'], 
        rotation=45, 
        ha='right',
        fontsize=10
    )
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # 标记峰值和低谷（修正坐标）
    if not monthly_mean.empty:
        peak_month = monthly_mean.idxmax()
        valley_month = monthly_mean.idxmin()
        
        # 峰值标注
        ax.annotate(
            f"峰值: {peak_month}月", 
            xy=(peak_month - 1, monthly_mean[peak_month]), 
            xytext=(0, 10), 
            textcoords='offset points',
            arrowprops=dict(arrowstyle='->'), 
            fontsize=10,
            ha='center'
        )
        
        # 低谷标注
        ax.annotate(
            f"低谷: {valley_month}月", 
            xy=(valley_month - 1, monthly_mean[valley_month]), 
            xytext=(0, -20), 
            textcoords='offset points',
            arrowprops=dict(arrowstyle='->'), 
            fontsize=10,
            ha='center'
        )

# 调整布局（适配4个国家）
plt.suptitle('多国图像使用频率季节性对比', fontsize=20, y=1.02)
plt.tight_layout()
plt.savefig('multi_seasonal_fixed.png', dpi=300)
plt.show()

In [ ]:
# 定义img_perc区间（保持原有逻辑）
country_img_stats['img_perc_bin'] = pd.cut(country_img_stats['img_perc_mean'], 
                                          bins=[0, 0.1, 0.2, 0.3, 0.4, 1],
                                          labels=['0-0.1', '0.1-0.2', '0.2-0.3', '0.3-0.4', '0.4+'])

# 统计各区间的国家数量并排序
bin_counts = country_img_stats['img_perc_bin'].value_counts().sort_index().reset_index()
bin_counts.columns = ['区间', '国家数量']  # 重命名列，便于后续使用

# 自定义颜色映射（从蓝色到红色的渐变色）
colors = sns.color_palette("Blues", len(bin_counts))[1:]  # 避免过浅的颜色

# 创建画布
plt.figure(figsize=(12, 7))

# 绘制柱状图（优化视觉效果）
bars = plt.bar(
    bin_counts['区间'], 
    bin_counts['国家数量'], 
    color=colors, 
    width=0.6,  # 调整柱子宽度
    edgecolor='black',  # 添加边框
    linewidth=1.2,  # 边框宽度
    zorder=3  # 确保在网格上方
)

# 在柱子上方添加数值标签
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2.,  # x坐标：柱子中心
        height + 0.5,  # y坐标：柱子上方0.5单位
        f'{height}',  # 显示数值
        ha='center',  # 水平居中
        va='bottom',  # 垂直底部对齐
        fontsize=12,  # 字体大小
        fontweight='bold'  # 字体加粗
    )

# 计算并添加统计摘要
total_countries = bin_counts['国家数量'].sum()
high_usage_countries = bin_counts[bin_counts['区间'] == '0.4+']['国家数量'].values[0]
high_usage_percentage = f"{high_usage_countries / total_countries * 100:.1f}%"

plt.figtext(
    0.5, 0.01,  # 位置：底部中央
    f"注：共有 {total_countries} 个国家参与统计，其中图像密度均值≥0.4的国家占比为 {high_usage_percentage}",
    ha="center",
    fontsize=10,
    bbox={"facecolor":"white", "alpha":0.8, "pad":5}
)

# 设置图表样式
plt.title('全球国家图像使用频率分布区间', fontsize=20, pad=20)
plt.xlabel('图像密度均值区间', fontsize=14, labelpad=10)
plt.ylabel('国家数量', fontsize=14, labelpad=10)

# 优化y轴显示（从0开始，调整刻度）
plt.ylim(0, bin_counts['国家数量'].max() * 1.1)  # 留出顶部空间
plt.yticks(fontsize=12)

# 网格优化
plt.grid(axis='y', linestyle='--', alpha=0.5, zorder=0)  # 网格置后

# 移除边框，保留底部边框
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_linewidth(1.2)  # 加粗底部边框

# 优化x轴标签
plt.xticks(fontsize=12, rotation=0)

# 增强整体美观度
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # 调整布局，为底部文本留出空间
plt.savefig('img_perc_bin_distribution_optimized.png', dpi=300, bbox_inches='tight')
plt.show()

### （二）分析点2：图像可用性（img_avail_perc）评估与异常检测

In [ ]:
# ---------------------- 1. 异常低值国家识别与筛选 ----------------------
# 计算 img_avail_perc 的 10% 分位数
q10 = df_ab['img_avail_perc'].quantile(0.1)  
print(f"img_avail_perc 的 10% 分位数为: {q10:.4f}")

# 筛选异常低值国家并过滤无意义值
abnormal_countries = df_ab[df_ab['img_avail_perc'] < q10]['country'].dropna().unique()
abnormal_countries = [country for country in abnormal_countries if country != 'UNKNOWN']  
print(f"图像可用性异常低值国家（筛选后）: {abnormal_countries}")

# 手动定义有分析价值的国家（示例：选取5个典型国家）
meaningful_countries = ['CN', 'US', 'FR', 'RU', 'IN']
representative_abnormal_countries = [c for c in meaningful_countries if c in abnormal_countries]

# 若指定国家不在异常列表中，从异常列表中选取前5个
if not representative_abnormal_countries:
    representative_abnormal_countries = abnormal_countries[:5]
print(f"最终用于可视化的代表性国家: {representative_abnormal_countries}")

# ---------------------- 2. 时间序列分析可视化 ----------------------
# 确保日期列是datetime类型
df_ab['date'] = pd.to_datetime(df_ab['date'])

# 定义颜色列表
colors = ['red', 'blue', 'yellow', 'green', 'orange']
plt.figure(figsize=(14, 7))

for idx, country in enumerate(representative_abnormal_countries):
    country_data = df_ab[df_ab['country'] == country].dropna(subset=['date', 'img_avail_perc'])
    country_data = country_data.sort_values('date')
    if not country_data.empty:
        # 绘制时间序列线
        sns.lineplot(
            x='date', y='img_avail_perc', 
            data=country_data, 
            label=country, 
            color=colors[idx % len(colors)],  
            alpha=0.8,
            linewidth=2.0
        )

# 添加关键日期（示例日期，可替换为真实事件日期）
key_dates = [
    pd.to_datetime('2013-11-10'),
    pd.to_datetime('2014-03-10'),
    pd.to_datetime('2015-08-01'),
]

# 确保图表的x轴使用日期格式
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # 每3个月显示一次

for key_date in key_dates:
    plt.axvline(
        x=key_date, 
        color='gray', linestyle='--', 
        alpha=0.5, 
        label=f'关键节点' if idx == 0 else None
    )

# 美化图表
plt.title('异常国家图像可用性时间序列', fontsize=14)
plt.xlabel('日期', fontsize=12)
plt.ylabel('图像可用性百分比', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.legend(fontsize=11)
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

# ---------------------- 3. 可用性差异对比分析可视化 ----------------------
# 计算可用性差值并分组汇总
df_ab['diff_avail'] = df_ab['img_avail_perc'] - df_ab['twt_img_avail_perc']
country_diff_mean = df_ab.groupby('country')['diff_avail'].mean().reset_index()

# 筛选代表性国家
representative_countries_diff = representative_abnormal_countries

# 绘制柱状图
plt.figure(figsize=(10, 5))
sns.barplot(
    x='country', 
    y='diff_avail', 
    data=country_diff_mean[country_diff_mean['country'].isin(representative_countries_diff)],
    palette=colors[:len(representative_countries_diff)],
    alpha=0.7
)
plt.title('外部链接与Twitter自带图像可用性差异对比', fontsize=13)
plt.xlabel('国家', fontsize=11)
plt.ylabel('可用性差值（外部-自带）', fontsize=11)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

### （三）分析点3：每日推文数量（num_tweet）的活跃度时间序列分析

In [ ]:
df = df_cd

# 定义目标国家（覆盖不同地区、文化和语言）
target_countries = ['CN', 'US', 'IN', 'GB', 'BR']
print(f"分析目标国家：{target_countries}")

# 筛选目标国家数据并转换日期格式
country_data = df[df['country'].isin(target_countries)]
country_data['date'] = pd.to_datetime(country_data['date'])
country_data = country_data.sort_values(['country', 'date'])

# 计算7日移动平均，平滑短期波动
country_data['num_tweet_ma7'] = country_data.groupby('country')['num_tweet'].rolling(window=7).mean().reset_index(drop=True)

# 打印数据概览
print("\n数据基本信息：")
print(country_data.info())
print("\n数据示例：")
print(country_data.head())


# ---------------------- 1. matplotlib静态时间序列图 ----------------------
# 设置图片清晰度及中文字体
plt.rcParams['figure.dpi'] = 300
plt.rcParams["font.family"] = ["SimHei"]

# 定义高对比度颜色方案
colors = {
    'CN': '#ff2501',  # 中国：红色
    'US': '#00A1FF',  # 美国：蓝色
    'IN': '#5ed935',  # 印度：绿色
    'GB': '#FFD100',  # 英国：黄色
    'BR': '#34B399'   # 巴西：青绿色
}

# 创建画布
plt.figure(figsize=(16, 9))

# 循环绘制各国数据
for country in target_countries:
    data = country_data[country_data['country'] == country]
    # 绘制原始数据（半透明线条）
    plt.plot(
        data['date'], data['num_tweet'],
        label=f'{country}(原始数据)',
        color=colors[country], alpha=0.6, zorder=2
    )
    # 绘制7日移动平均（加粗线条）
    plt.plot(
        data['date'], data['num_tweet_ma7'],
        label=f'{country}(7日移动平均)',
        color=colors[country], linewidth=2, zorder=3
    )

# 美化图表
plt.title('中、美、印、英、巴五国每日推文数量时间序列', fontsize=18)
plt.xlabel('日期', fontsize=14)
plt.ylabel('推文数量', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()


# ---------------------- 2. pyecharts动态时间轴图表（修正版）----------------------
from pyecharts.charts import Timeline, Line
from pyecharts import options as opts
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 定义2013-2023年间真实重大事件（覆盖全球范围）
real_events = [
    {"date": "2016-06-23", "event": "英国脱欧公投"},
    {"date": "2016-08-05", "event": "里约奥运会开幕"},
    {"date": "2018-07-15", "event": "俄罗斯世界杯决赛"},
    {"date": "2019-05-23", "event": "印度大选结果公布"},
    {"date": "2020-01-20", "event": "新冠疫情人传人确认"},
    {"date": "2020-11-03", "event": "2020年美国大选投票日"},
    {"date": "2021-07-23", "event": "东京奥运会开幕（延期）"},
    {"date": "2022-11-20", "event": "卡塔尔世界杯开幕"}
]

# 创建时间轴图表对象
timeline = Timeline(init_opts=opts.InitOpts(width="1200px", height="700px"))
timeline.add_schema(
    orient="vertical",          # 垂直时间轴
    is_auto_play=True,          # 自动播放
    play_interval=5000,         # 播放间隔5秒
    label_opts=opts.LabelOpts(position="right")  # 标签显示在右侧
)

# 循环添加各国时间线
for country in target_countries:
    data = country_data[country_data['country'] == country].sort_values('date')
    date_list = data['date'].dt.strftime("%Y-%m-%d").tolist()
    
    # 准备数据点 - 转换为列表格式
    tweet_values = data['num_tweet'].tolist()
    ma_values = data['num_tweet_ma7'].tolist()
    
    # 创建折线图
    line = Line()
    line.add_xaxis(date_list)
    
    # 添加事件标注
    event_data = []
    for event in real_events:
        if event["date"] in date_list:
            # 安全获取索引位置
            idx = date_list.index(event["date"])
            # 使用移动平均值作为y坐标（更平滑）
            y_value = ma_values[idx] if idx < len(ma_values) else tweet_values[idx]
            event_data.append(
                opts.MarkPointItem(
                    name=event["event"],
                    coord=[event["date"], y_value],
                    value=event["event"],
                    symbol="pin",
                    symbol_size=30,
                    itemstyle_opts=opts.ItemStyleOpts(color="red"),
                )
            )
    
    # 添加第一个系列（推文数量）并包含事件标注
    line.add_yaxis(
        series_name="推文数量",
        y_axis=tweet_values,
        is_smooth=True,
        label_opts=opts.LabelOpts(is_show=False),
        linestyle_opts=opts.LineStyleOpts(width=2, opacity=0.7),
        markpoint_opts=opts.MarkPointOpts(data=event_data) if event_data else None
    )
    
    # 添加第二个系列（7日移动平均）
    line.add_yaxis(
        series_name="7日移动平均",
        y_axis=ma_values,
        is_smooth=True,
        label_opts=opts.LabelOpts(is_show=False),
        linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.9)
    )
    
    # 设置全局选项
    line.set_global_opts(
        title_opts=opts.TitleOpts(title=f"{country}每日推文数量时间序列"),
        xaxis_opts=opts.AxisOpts(name="日期", type_="category"),
        yaxis_opts=opts.AxisOpts(name="推文数量"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
        legend_opts=opts.LegendOpts(pos_top="5%"),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        datazoom_opts=[opts.DataZoomOpts()]  # 添加缩放功能
    )
    
    # 将当前国家的时间线添加到总时间轴
    timeline.add(line, time_point=country)

# 渲染图表
try:
    # Jupyter环境
    from IPython.display import display, HTML
    display(HTML(timeline.render_embed()))
except:
    # 非Jupyter环境
    timeline.render("tweet_activity_timeline.html")
    print("动态图表已保存为 tweet_activity_timeline.html")

### （四）分析点4：用户参与度分析（人均推文数）

In [ ]:
# 设置中文字体
font = FontProperties(fname=r"C:\Windows\Fonts\simhei.ttf")  # 根据实际情况修改字体路径
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

df = df_cd
# 去除 UNKNOWN 国家的数据
df = df[df['country'] != 'UNKNOWN']

# 1. 计算人均推文数
df['per_capita_tweet'] = df['num_tweet'] / df['num_user']

# 2. 按国家分组，计算人均推文数的均值
country_per_capita = df.groupby('country')['per_capita_tweet'].mean().reset_index()

# 3. 识别高参与度国家（人均推文数>10，可根据实际情况调整）
high_engagement_countries = country_per_capita[country_per_capita['per_capita_tweet'] > 10]['country'].tolist()
print(f"高参与度国家（人均推文数>10）: {high_engagement_countries}")

# 4. 选择 10 个重要有分析价值的国家进行分析
important_countries = ['US', 'CN', 'GB', 'JP', 'FR', 'DE', 'IN', 'BR', 'CA', 'AU']
df_selected = df[df['country'].isin(important_countries)]

# 5. 绘制各国人均推文数分布箱线图
plt.figure(figsize=(14, 8))
sns.boxplot(x='country', y='per_capita_tweet', data=df_selected, palette='viridis')
plt.title('10 个重要国家人均推文数分布', fontproperties=font, fontsize=16)
plt.xlabel('国家', fontproperties=font, fontsize=13)
plt.ylabel('人均推文数', fontproperties=font, fontsize=13)
plt.xticks(fontproperties=font)
# 可根据数据情况微调 y 轴范围，让数据展示更合理
plt.ylim(0, df_selected['per_capita_tweet'].quantile(0.95) * 1.2)  
plt.tight_layout()
plt.savefig('人均推文数分布箱线图.png', dpi=300)
plt.show()

# 6. 绘制人均推文数与语言种类数的散点图，优化点的大小、透明度和配色等
plt.figure(figsize=(12, 8))
sns.scatterplot(x='num_lang', y='per_capita_tweet', hue='country', data=df_selected, 
                palette='tab10',  # 更丰富清晰的配色
                s=60,  # 调整点的大小
                alpha=0.7,  # 调整透明度，平衡重叠和清晰度
                edgecolor='white',  # 给点添加白色边缘，增强区分度
                linewidth=0.5)
plt.title('10 个重要国家人均推文数与语言种类数的关系', fontproperties=font, fontsize=16)
plt.xlabel('语言种类数', fontproperties=font, fontsize=13)
plt.ylabel('人均推文数', fontproperties=font, fontsize=13)
plt.legend(title='国家', prop=font, bbox_to_anchor=(1.02, 1), loc='upper left')  # 图例放右侧，避免遮挡
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig('人均推文数与语言种类数散点图.png', dpi=300, bbox_inches='tight')
plt.show()

# 7. 找出人均推文数最高的 3 个国家
top3_countries = country_per_capita.sort_values(by='per_capita_tweet', ascending=False).head(3)['country'].tolist()
df_top3 = df[df['country'].isin(top3_countries)]

# 8. 绘制人均推文数最高的 3 个国家的人均推文数与语言种类数的关系散点图
plt.figure(figsize=(10, 6))
sns.scatterplot(x='num_lang', y='per_capita_tweet', hue='country', data=df_top3, 
                palette='Set1', 
                s=80, 
                alpha=0.8, 
                edgecolor='black', 
                linewidth=1)
plt.title(f'人均推文数最高的 3 个国家人均推文数与语言种类数的关系', fontproperties=font, fontsize=15)
plt.xlabel('语言种类数', fontproperties=font, fontsize=12)
plt.ylabel('人均推文数', fontproperties=font, fontsize=12)
plt.legend(title='国家', prop=font)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('top3_countries_scatter.png', dpi=300, bbox_inches='tight')
plt.show()

# 9. 检查用户数较少但推文数较高的国家（可能存在机器人或大V）
small_user_high_tweet = df[(df['num_user'] < 100) & (df['num_tweet'] > 1000)]
if not small_user_high_tweet.empty:
    print("\n可能存在机器人或大V的国家分析：")
    for _, row in small_user_high_tweet.head().iterrows():
        print(f"国家: {row['country']}, 日期: {row['date']}, 推文数: {row['num_tweet']}, 用户数: {row['num_user']}, 人均推文数: {row['per_capita_tweet']:.2f}")
else:
    print("\n未发现用户数较少但推文数较高的异常情况")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pycountry
from matplotlib.font_manager import FontProperties
import matplotlib as mpl

# 设置中文字体
font = FontProperties(fname=r"C:\Windows\Fonts\simhei.ttf")
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'SimHei'  # 全局设置中文字体
mpl.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 设置中文字体
plt.rcParams["font.family"] = ["SimHei"]
plt.rcParams['axes.unicode_minus'] = False
rc = {'font.sans-serif': 'SimHei','axes.unicode_minus': False}
sns.set(context='notebook', style='ticks', rc=rc, font_scale=1.2)

# 读取数据（这里假设df_cd是已经加载的DataFrame）
# 如果从文件读取，使用：df = pd.read_csv('TH_country_day.csv')
df = df_cd.copy()

# 数据预处理
print("原始数据形状:", df.shape)

# 1. 日期转换
df['date'] = pd.to_datetime(df['date'])

# 2. 去除UNKNOWN国家的数据
df = df[df['country'] != 'UNKNOWN']
print("去除UNKNOWN国家后数据形状:", df.shape)

# 3. 计算人均推文数
df['per_capita_tweet'] = df['num_tweet'] / df['num_user']

# 4. 处理无穷大和空值
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=['per_capita_tweet'])
print("处理缺失值后数据形状:", df.shape)

# 按国家计算平均指标
country_stats = df.groupby('country').agg(
    avg_per_capita=('per_capita_tweet', 'mean'),
    total_tweets=('num_tweet', 'sum'),
    total_users=('num_user', 'sum'),
    lang_diversity=('num_lang', 'mean')
).reset_index()

# 国家代码转换函数
def iso2_to_iso3(iso2_code):
    """将ISO 3166-1 alpha-2代码转换为alpha-3代码"""
    try:
        country = pycountry.countries.get(alpha_2=iso2_code)
        return country.alpha_3
    except (AttributeError, LookupError):
        # 处理特殊地区代码
        special_cases = {
            'TW': 'TWN',  # 台湾
            'HK': 'HKG',  # 香港
            'MO': 'MAC',  # 澳门
            'XK': 'XKX',  # 科索沃
            'PS': 'PSE'   # 巴勒斯坦
        }
        return special_cases.get(iso2_code, None)

# 应用国家代码转换
country_stats['iso3'] = country_stats['country'].apply(iso2_to_iso3)

# 检查未识别的国家代码
unrecognized = country_stats[country_stats['iso3'].isnull()]
if not unrecognized.empty:
    print("\n未识别的国家代码:")
    print(unrecognized['country'].unique())
    print(f"共 {len(unrecognized)} 个国家/地区未识别")
    
    # 处理未识别的国家：使用原始代码并添加后缀
    country_stats.loc[country_stats['iso3'].isnull(), 'iso3'] = unrecognized['country'] + '_XX'

# 创建世界地图热力图
fig = px.choropleth(
    country_stats,
    locations='iso3',                # 使用三字母代码
    locationmode='ISO-3',            # 指定使用ISO-3格式
    color='avg_per_capita',
    color_continuous_scale='YlOrRd',
    hover_name='country',
    hover_data={
        'avg_per_capita': ':.2f',
        'total_tweets': ':,.0f',
        'total_users': ':,.0f',
        'lang_diversity': ':.1f',
        'iso3': False  # 不在悬停中显示
    },
    projection='natural earth',
    title='全球人均推文数分布热力图',
    labels={'avg_per_capita': '人均推文数'}
)

# 优化地图样式
fig.update_geos(
    showcountries=True,
    countrycolor="LightGrey",
    showocean=True,
    oceancolor="LightCyan",
    resolution=110  # 更高分辨率
)

fig.update_layout(
    height=700,
    margin={"r": 0, "t": 60, "l": 0, "b": 0},
    coloraxis_colorbar=dict(
        title='人均推文数',
        tickvals=np.linspace(
            country_stats['avg_per_capita'].min(), 
            country_stats['avg_per_capita'].max(), 
            5
        ),  # 动态设置刻度
        tickformat=".1f"
    )
)

# 显示地图
fig.show()
print("地理热力图已生成")

# 保存为HTML文件以便分享
fig.write_html("global_tweet_density_map.html")
print("地理热力图已保存为HTML文件")

# 分析高参与度国家
# 识别人均推文数前10的国家
top10_countries = country_stats.nlargest(10, 'avg_per_capita').sort_values('avg_per_capita', ascending=True)

plt.figure(figsize=(12, 8))
sns.barplot(x='avg_per_capita', y='country', data=top10_countries, palette='viridis')
plt.title('人均推文数最高的10个国家', fontsize=16, fontproperties=font)
plt.xlabel('人均推文数', fontsize=13, fontproperties=font)
plt.ylabel('国家', fontsize=13, fontproperties=font)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('top10_countries.png', dpi=300, bbox_inches='tight')
plt.show()
print("TOP10国家柱状图已生成")

# 时间序列分析：选取代表性国家观察变化趋势
selected_countries = ['US', 'GB', 'JP', 'IN', 'BR', 'RU', 'CN', 'DE', 'FR', 'CA']
df_selected = df[df['country'].isin(selected_countries)]

# 计算季度平均
df_selected['quarter'] = df_selected['date'].dt.to_period('Q')
quarterly_stats = df_selected.groupby(['country', 'quarter']).agg(
    avg_per_capita=('per_capita_tweet', 'mean'),
    total_tweets=('num_tweet', 'sum')
).reset_index()
quarterly_stats['quarter'] = quarterly_stats['quarter'].astype(str)

# 创建时间序列图
plt.figure(figsize=(16, 10))
sns.set_style("whitegrid")  # 设置网格样式
palette = sns.color_palette("husl", len(selected_countries))  # 使用更鲜明的调色板

# 绘制线图
sns.lineplot(
    x='quarter', 
    y='avg_per_capita', 
    hue='country', 
    data=quarterly_stats, 
    marker='o', 
    linewidth=2.5,
    palette=palette,
    markersize=8
)

plt.title('代表性国家人均推文数季度变化趋势', fontsize=18, fontproperties=font)
plt.xlabel('季度', fontsize=14, fontproperties=font)
plt.ylabel('人均推文数', fontsize=14, fontproperties=font)
plt.legend(
    title='ISO 3155-2', 
    fontsize=10, 
    title_fontsize=12,
    loc='upper left', 
    bbox_to_anchor=(1, 1)
)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig('quarterly_trends.png', dpi=300, bbox_inches='tight')
plt.show()
print("时间序列图已生成")

# 相关性分析
corr_df = country_stats[['avg_per_capita', 'lang_diversity', 'total_users']].copy()
corr_df.columns = ['人均推文数', '语言多样性', '总用户数']  # 使用中文列名

plt.figure(figsize=(12, 10))
sns.heatmap(
    corr_df.corr(),
    annot=True, 
    cmap='coolwarm', 
    fmt=".2f", 
    linewidths=0.5,
    annot_kws={"size": 12}
)
plt.title('国家层面指标相关性分析', fontsize=18, fontproperties=font)
plt.xticks(fontsize=12, fontproperties=font)
plt.yticks(fontsize=12, fontproperties=font)
plt.tight_layout()
plt.savefig('correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()
print("相关性热力图已生成")

# 附加分析：用户参与度与语言多样性的关系
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x='lang_diversity', 
    y='avg_per_capita', 
    size='total_users', 
    sizes=(20, 400),
    alpha=0.7,
    hue='total_tweets',
    palette='viridis',
    data=country_stats
)
plt.title('用户参与度与语言多样性的关系', fontsize=16, fontproperties=font)
plt.xlabel('平均语言多样性', fontsize=13, fontproperties=font)
plt.ylabel('人均推文数', fontsize=13, fontproperties=font)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(title='总推文量', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('scatter_lang_engagement.png', dpi=300, bbox_inches='tight')
plt.show()
print("散点图已生成")

# 附加分析：异常值检测（机器人/大V账户）
# 识别用户数少但推文量高的异常情况
df['user_activity'] = df['num_tweet'] / df['num_user']
high_activity = df[(df['num_user'] < 100) & (df['num_tweet'] > 1000)]

if not high_activity.empty:
    print("\n检测到异常活动（可能为机器人或大V账户）：")
    print(f"共发现 {len(high_activity)} 条记录")
    
    # 按国家分组统计异常情况
    anomaly_by_country = high_activity.groupby('country').agg(
        anomaly_count=('date', 'count'),
        avg_tweets_per_user=('user_activity', 'mean'),
        max_tweets=('num_tweet', 'max')
    ).sort_values('anomaly_count', ascending=False).reset_index()
    
    # 绘制异常活动国家分布
    plt.figure(figsize=(12, 8))
    sns.barplot(
        x='anomaly_count', 
        y='country', 
        data=anomaly_by_country.head(10),
        palette='rocket'
    )
    plt.title('异常活动最多的10个国家', fontsize=16, fontproperties=font)
    plt.xlabel('异常活动次数', fontsize=13, fontproperties=font)
    plt.ylabel('国家', fontsize=13, fontproperties=font)
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('anomaly_countries.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 绘制异常活动的时间分布
    high_activity['year'] = high_activity['date'].dt.year
    anomaly_by_year = high_activity.groupby('year').size().reset_index(name='count')
    
    plt.figure(figsize=(10, 6))
    sns.lineplot(
        x='year', 
        y='count', 
        data=anomaly_by_year,
        marker='o',
        linewidth=2.5
    )
    plt.title('异常活动年度分布', fontsize=16, fontproperties=font)
    plt.xlabel('年份', fontsize=13, fontproperties=font)
    plt.ylabel('异常活动次数', fontsize=13, fontproperties=font)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('anomaly_trend.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("\n未发现用户数较少但推文量较高的异常情况")

print("\n所有分析已完成！")

### （五）分析点5：语言多样性（num_lang）的跨国对比

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Map
from pyecharts.globals import ThemeType
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib.font_manager import FontProperties

# 设置中文字体
font = FontProperties(fname=r"C:\Windows\Fonts\simhei.ttf")  # 根据实际情况修改字体路径
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 假设df_day已加载TH_country_day.csv，df_lang_day已加载TH_country_lang_day.csv
df_day = df_cd
df_lang_day = df_cld

# 1. 计算各国语言多样性均值（num_lang 均值）
country_lang_mean = df_day.groupby('country')['num_lang'].mean().reset_index()
country_lang_mean.columns = ['country', 'avg_num_lang']

# 2. 定义多语种和单一语种国家（示例）
multilingual_countries = ['IN', 'CA', 'US', 'GB', 'ZA']  
monolingual_countries = ['JP', 'KR', 'DE', 'FR', 'IT']  

# 3. 语言多样性最高的 10 个国家
top10_lang_countries = country_lang_mean.sort_values(by='avg_num_lang', ascending=False).head(10)
print("语言多样性最高的 10 个国家：")
print(top10_lang_countries)

# 4. 绘制语言多样性热力图（pyecharts）
country_name_map = {
    'IN': '印度', 'CA': '加拿大', 'US': '美国', 'GB': '英国', 'ZA': '南非',
    'JP': '日本', 'KR': '韩国', 'DE': '德国', 'FR': '法国', 'IT': '意大利',
    'CN': '中国', 'RU': '俄罗斯', 'BR': '巴西', 'AU': '澳大利亚'
}
map_data = [
    (country_name_map[country], float(avg_lang)) 
    for country, avg_lang in country_lang_mean[country_lang_mean['country'].isin(country_name_map.keys())][['country', 'avg_num_lang']].values
]

map_fig = (
    Map(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, width="1200px", height="600px"))
    .add("语言多样性指数", map_data, "world", is_map_symbol_show=False)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="全球语言多样性热力图"),
        visualmap_opts=opts.VisualMapOpts(
            min_=1, max_=20, range_text=["高", "低"], 
            range_color=["lightskyblue", "yellow", "orangered"]
        ),
        toolbox_opts=opts.ToolboxOpts(is_show=True)
    )
)
map_fig.render("language_diversity_heatmap.html")

# 5. 对比多语种与单一语种国家语言多样性（箱线图 + 小提琴图）
plt.figure(figsize=(14, 6))
df_multi = df_day[df_day['country'].isin(multilingual_countries)]
df_mono = df_day[df_day['country'].isin(monolingual_countries)]
df_compare = pd.concat([df_multi, df_mono])

# 箱线图（左）
plt.subplot(1, 2, 1)
sns.boxplot(x='country', y='num_lang', data=df_compare, palette='Set1')
plt.title('多语种与单一语种国家语言多样性对比（箱线图）', fontproperties=font, fontsize=13)
plt.xlabel('国家', fontproperties=font, fontsize=11)
plt.ylabel('语言种类数', fontproperties=font, fontsize=11)
plt.xticks( fontproperties=font)

# 小提琴图（右）
plt.subplot(1, 2, 2)
sns.violinplot(x='country', y='num_lang', data=df_compare, palette='Set2', bw=0.5)
plt.title('多语种与单一语种国家语言多样性对比（小提琴图）', fontproperties=font, fontsize=13)
plt.xlabel('国家', fontproperties=font, fontsize=11)
plt.ylabel('语言种类数', fontproperties=font, fontsize=11)
plt.xticks( fontproperties=font)

plt.tight_layout()
plt.savefig('language_diversity_comparison.png', dpi=300)
plt.show()

# 6. 印度推文中主要语言占比（优化版饼图）
# 提取印度的语言数据
india_data = df_lang_day[df_lang_day['country'] == 'IN']
# 计算各语言的推文总数
india_lang_sum = india_data.groupby('lang')['n_tweets'].sum().reset_index()
# 计算占比
india_lang_sum['占比'] = india_lang_sum['n_tweets'] / india_lang_sum['n_tweets'].sum()
# 按占比降序排列并筛选主要语言（示例：前10种）
india_lang_sum = india_lang_sum.sort_values(by='占比', ascending=False).head(10)

# 定义颜色列表（可根据需要调整）
colors_india = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2c2f0', '#ff99c2', '#99ffff', '#ffcc99', '#c2f0c2', '#f0c2f0']

plt.figure(figsize=(10, 10))
ax = plt.subplot(111)

# 只显示占比>5%的标签
def custom_autopct(pct):
    return f'{pct:.1f}%' if pct > 5 else ''

wedges, texts, autotexts = plt.pie(
    india_lang_sum['占比'],
    startangle=90,
    colors=colors_india,
    wedgeprops={'linewidth': 1, 'edgecolor': 'white'},
    labels=None,  # 移除直接标签
    autopct=custom_autopct,  # 只显示大占比标签
    pctdistance=0.85,  # 百分比标签位置
    textprops={'fontproperties': font, 'fontsize': 14}
)

# 创建带引导线的图例
legend_labels = [f"{lang} ({p*100:.1f}%)" 
                for lang, p in zip(india_lang_sum['lang'], india_lang_sum['占比'])]

# 将图例放在图表下方（水平排列）
ax.legend(wedges, legend_labels,
          title='语言分布',
          loc='lower center',
          bbox_to_anchor=(0.5, -0.1),
          ncol=min(4, len(india_lang_sum)),  # 最大4列
          fontsize=14,
          title_fontproperties=font,
          prop=font,
          frameon=False)

plt.title('印度推文语言分布', fontproperties=font, fontsize=15, pad=20)
plt.axis('equal')
plt.tight_layout()
plt.subplots_adjust(bottom=0.25)  # 为底部图例留空间
plt.savefig('india_language_piechart.png', dpi=300, bbox_inches='tight')
plt.show()

# 7. 加拿大推文中主要语言占比（优化版环形图）
# 提取加拿大的语言数据
ca_data = df_lang_day[df_lang_day['country'] == 'CA']
# 计算各语言的推文总数
ca_lang_sum = ca_data.groupby('lang')['n_tweets'].sum().reset_index()
# 计算占比
ca_lang_sum['占比'] = ca_lang_sum['n_tweets'] / ca_lang_sum['n_tweets'].sum()
# 按占比降序排列并筛选主要语言（示例：前10种）
ca_lang_sum = ca_lang_sum.sort_values(by='占比', ascending=False).head(10)

# 定义颜色列表（可根据需要调整）
colors_ca = ['#ff6666', '#66b3ff', '#99ff99', '#ffcc99', '#99ccff', '#ff99cc', '#66ff99', '#ffcc99', '#cc99ff', '#99ffcc']

plt.figure(figsize=(10, 10))
ax = plt.subplot(111)

wedges, texts, autotexts = plt.pie(
    ca_lang_sum['占比'],
    startangle=90,
    colors=colors_ca,
    wedgeprops={'width': 0.3, 'edgecolor': 'white', 'linewidth': 1},  # 环形图通过width参数实现
    labels=None,  # 移除直接标签
    autopct=custom_autopct,  # 只显示大占比标签
    pctdistance=0.75,  # 百分比标签位置（更靠近中心）
    textprops={'fontproperties': font, 'fontsize': 14}
)

# 创建加拿大图例
legend_labels_ca = [f"{lang} ({p*100:.1f}%)" 
                   for lang, p in zip(ca_lang_sum['lang'], ca_lang_sum['占比'])]

# 将图例放在图表下方（水平排列）
ax.legend(wedges, legend_labels_ca,
          title='语言分布',
          loc='lower center',
          bbox_to_anchor=(0.5, -0.1),
          ncol=min(3, len(ca_lang_sum)),  # 最大3列
          fontsize=14,
          title_fontproperties=font,
          prop=font,
          frameon=False)

plt.title('加拿大推文语言分布', fontproperties=font, fontsize=15, pad=20)
plt.axis('equal')
plt.tight_layout()
plt.subplots_adjust(bottom=0.25)  # 为底部图例留空间
plt.savefig('ca_language_ringchart.png', dpi=300, bbox_inches='tight')
plt.show()

### （六）分析点6：新增用户数（num_new_user）的时间趋势对比

In [ ]:
from plotly.express import line, bar

# 假设nnu_df已加载TH_num_new_user.csv数据
nnu_df = df_nnu

# 补充ISO 3166-1标准中部分国家和地区信息
country_info = {
    'US': '美国', 'CN': '中国', 'IN': '印度', 'BR': '巴西', 'DE': '德国', 'JP': '日本',
    'RU': '俄罗斯', 'FR': '法国', 'GB': '英国', 'AU': '澳大利亚', 'CA': '加拿大'
}
nnu_df['country_name'] = nnu_df['country'].map(country_info)

# 筛选重要国家数据
important_countries = ['US', 'CN', 'IN', 'BR', 'DE', 'JP']
filtered_nnu_df = nnu_df[nnu_df['country'].isin(important_countries)]

# 按国家、国家名称和时间分组，计算每日新增用户数总和
grouped_nnu = filtered_nnu_df.groupby(['country', 'country_name', 'creation_date'])['num_new_user'].sum().reset_index()

# 绘制时间序列趋势图
fig = line(
    grouped_nnu, 
    x='creation_date', 
    y='num_new_user', 
    color='country_name',  # 确保color列存在于数据框中
    title='重要国家新增用户数时间序列趋势图',
    labels={
        'creation_date': '创建日期', 
        'num_new_user': '每日新增用户数', 
        'country_name': '国家'
    }
)

# 配置交互功能（时间范围选择器、滑块）
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label='7天', step='day', stepmode='backward'),
                dict(count=30, label='30天', step='day', stepmode='backward'),
                dict(count=90, label='90天', step='day', stepmode='backward'),
                dict(count=1, label='1年', step='year', stepmode='backward'),
                dict(step='all', label='全部')
            ])
        ),
        rangeslider=dict(visible=True),
        type='date'
    ),
    # 优化图例显示
    legend=dict(
        title='国家',
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    width=1000,
    height=600
)

fig.show()

# 计算重要国家累计新增用户数
cumulative_users = filtered_nnu_df.groupby(['country', 'country_name'])['num_new_user'].sum().reset_index()

# 绘制累计新增用户数对比图
fig = bar(
    cumulative_users, 
    x='country_name', 
    y='num_new_user',
    title='重要国家累计新增用户数对比',
    labels={
        'country_name': '国家', 
        'num_new_user': '累计新增用户数'
    }
)

fig.update_layout(
    width=800,
    height=600
)

fig.show()

### （七）分析点7：事件驱动的新增用户波动检测

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# 设置中文字体
plt.rcParams["font.family"] = ["SimHei"]
plt.rcParams['axes.unicode_minus'] = False
rc = {'font.sans-serif': 'SimHei', 'axes.unicode_minus': False}
sns.set(context='notebook', style='ticks', rc=rc, font_scale=1.2)


# 1. 数据加载与预处理
df = df_nnu
df['creation_date'] = pd.to_datetime(df['creation_date'])

# 选择重点国家（社交媒体用户大国）
target_countries = ['US', 'GB', 'IN', 'JP', 'BR', 'NG']  # 美英印日巴西尼日利亚
df = df[df['country'].isin(target_countries)]


# 2. 时间序列处理
def process_country_data(country_code):
    country_df = df[df['country'] == country_code].copy()
    country_df.set_index('creation_date', inplace=True)
    country_df = country_df.resample('D').sum().fillna(0)  # 日级重采样

    # 计算移动平均和标准差
    country_df['30d_MA'] = country_df['num_new_user'].rolling(30).mean()
    country_df['30d_std'] = country_df['num_new_user'].rolling(30).std()
    country_df['upper'] = country_df['30d_MA'] + 2 * country_df['30d_std']
    country_df['lower'] = country_df['30d_MA'] - 2 * country_df['30d_std']

    # 标记异常点 (超出2个标准差)
    country_df['outlier'] = np.where(
        (country_df['num_new_user'] > country_df['upper']) |
        (country_df['num_new_user'] < country_df['lower']), 1, 0)

    return country_df


# 3. 可视化函数
def plot_country_volatility(country_df, country_name, country_code):
    plt.figure(figsize=(14, 8))

    # 主趋势线
    plt.plot(country_df.index, country_df['num_new_user'],
             alpha=0.3, color='steelblue', label='每日新增用户')

    # 移动平均线
    plt.plot(country_df.index, country_df['30d_MA'],
             'r-', linewidth=2, label='30日移动平均')

    # 波动区间填充
    plt.fill_between(country_df.index, country_df['lower'], country_df['upper'],
                     color='lightcoral', alpha=0.3, label='2σ波动区间')

    # 标注异常点
    outliers = country_df[country_df['outlier'] == 1]
    plt.scatter(outliers.index, outliers['num_new_user'],
                color='red', s=40, zorder=5, label='异常波动点')

    # 标注重大事件示例
    events = {
        '2009-06-11': '甲型H1N1流感流行',
        '2011-03-11': '日本大地震并引发海啸和核泄漏危机',
        '2013-03-23': '“人类命运共同体”理念',
        '2014-08-08': '埃博拉疫情构成国际公共卫生紧急事件',
        '2015-12-12': '《巴黎协定》达成',
        '2016-06-23': '英国举行脱欧公投',
        '2018-06-12': '朝美领导人新加坡会晤',
        '2020-03-11': '新冠疫情全球流行',
        '2021-01-06': '美国国会山骚乱事件',
        '2023-02-06': '土耳其南部地震'
    }
    for date_str, label in events.items():
        event_date = pd.Timestamp(date_str)
        if event_date in country_df.index:
            plt.axvline(event_date, color='purple', linestyle='--', alpha=0.7)
            plt.text(event_date, country_df['num_new_user'].max() * 0.9,
                     label, rotation=90, verticalalignment='top')

    plt.title(f'{country_name}新增用户波动检测', fontsize=16)
    plt.xlabel('日期', fontsize=12)
    plt.ylabel('新增用户数', fontsize=12)
    plt.legend()
    plt.grid(alpha=0.2)
    plt.tight_layout()
    plt.savefig(f'{country_code}_volatility.png', dpi=300)
    plt.show()


# 4. 年度波动热力图
def plot_annual_heatmap(country_df, country_name, country_code):
    # 提取年度/月度数据
    heatmap_data = country_df.copy()
    heatmap_data['year'] = heatmap_data.index.year
    heatmap_data['month'] = heatmap_data.index.month

    # 计算月度同比波动率
    monthly = heatmap_data.groupby(['year', 'month'])['num_new_user'].sum().unstack()
    monthly_pct = monthly.pct_change(axis=0) * 100  # 年度同比变化

    plt.figure(figsize=(12, 6))
    sns.heatmap(monthly_pct.T, cmap='coolwarm', center=0,
                annot=True, fmt=".0f", linewidths=.5,
                annot_kws={"size": 8}, cbar_kws={'label': '同比变化率(%)'})

    plt.title(f'{country_name}月度新增用户同比变化热力图', fontsize=14)
    plt.xlabel('年份')
    plt.ylabel('月份')
    plt.tight_layout()
    plt.savefig(f'{country_code}_annual_heatmap.png', dpi=300)
    plt.show()


# 5. 执行分析
country_names = {'US': '美国', 'IN': '印度', 'BR': '巴西'}

for code, name in country_names.items():
    country_data = process_country_data(code)
    plot_country_volatility(country_data, name, code)
    plot_annual_heatmap(country_data, name, code)

    # 生成统计摘要
    volatility_index = (country_data['30d_std'].mean() /
                        country_data['30d_MA'].mean()) * 100
    print(f"{name}波动性指数: {volatility_index:.1f}%")
    print(f"异常波动天数占比: {country_data['outlier'].mean():.2%}")

### （八）分析点8：活跃度与图像可用性的联合分析

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyecharts import options as opts
from pyecharts.charts import Line
from pyecharts.globals import ThemeType, CurrentConfig
from IPython.display import display, HTML
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# Pyecharts 在 Jupyter 中渲染的配置
CurrentConfig.ONLINE_HOST = "https://cdn.jsdelivr.net/npm/echarts/dist/"

# 假设已加载 IS_availability.csv 和 TH_country_day.csv 数据
# 1. 数据加载与预处理
try:
    # 读取活跃度数据（TH_country_day.csv）
    df_activity = df_cd
    # 读取图像可用性数据（IS_availability.csv）
    df_img = df_ab
    print("数据读取成功，基本信息：")
    print("活跃度数据：", df_activity.info())
    print("图像可用性数据：", df_img.info())
except FileNotFoundError:
    print("找不到数据文件，使用模拟数据进行演示...")
    np.random.seed(42)
    
    # 生成模拟活跃度数据（TH_country_day.csv）
    dates = pd.date_range(start='2015-01-01', end='2023-03-14', freq='D')
    countries_activity = ['US', 'CN', 'IN', 'GB', 'FR']
    data_activity = []
    for date in dates:
        for country in countries_activity:
            num_tweet = np.random.randint(1000, 100000)
            num_user = np.random.randint(100, 10000)
            num_lang = np.random.randint(1, 20)
            data_activity.append([date.strftime('%Y-%m-%d'), country, num_tweet, num_user, num_lang])
    df_activity = pd.DataFrame(data_activity, columns=['date', 'country', 'num_tweet', 'num_user', 'num_lang'])
    
    # 生成模拟图像可用性数据（IS_availability.csv）
    countries_img = ['US', 'CN', 'IN', 'GB', 'FR']
    data_img = []
    for date in dates:
        for country in countries_img:
            total_lines = np.random.randint(1000, 100000)
            total_urls = np.random.randint(500, total_lines)
            downloaded_images = np.random.randint(100, total_urls)
            img_perc = downloaded_images / total_lines if total_lines > 0 else 0
            img_avail_perc = downloaded_images / total_urls if total_urls > 0 else 0
            data_img.append([date.strftime('%Y-%m-%d'), country, total_lines, total_urls, 
                            downloaded_images, img_perc, img_avail_perc, 0, 0, 0, 0, 0, 0])
    df_img = pd.DataFrame(data_img, columns=['date', 'country', 'total_lines', 'total_urls', 
                                           'downloaded_images', 'img_perc', 'img_avail_perc',
                                           'downloaded_images_new', 'twt_img_avail_perc',
                                           'img_perc_ma', 'img_perc_sd', 'img_avail_perc_ma', 'img_avail_perc_sd'])

# 转换日期格式
df_activity['date'] = pd.to_datetime(df_activity['date'])
df_img['date'] = pd.to_datetime(df_img['date'])

# 2. 选择重点国家（增加有研究价值的国家，如 'CN', 'IN', 'GB'）
selected_countries = ['US', 'CN', 'IN', 'GB', 'FR']  # 美国、中国、印度、英国、法国
df_activity_selected = df_activity[df_activity['country'].isin(selected_countries)]
df_img_selected = df_img[df_img['country'].isin(selected_countries)]

# 3. 数据关联：合并活跃度与图像可用性数据
df_combined = pd.merge(
    df_activity_selected, 
    df_img_selected[['date', 'country', 'img_perc', 'img_avail_perc']],
    on=['date', 'country'],
    how='inner'
)

# 4. 计算活跃度指标：人均推文数
df_combined['per_capita_tweet'] = df_combined['num_tweet'] / df_combined['num_user']

# 5. 数据预处理：处理异常值
df_combined = df_combined[df_combined['per_capita_tweet'] > 0]
df_combined = df_combined.dropna(subset=['img_perc', 'img_avail_perc'])

# 6. 活跃度与图像可用性联合分析可视化

# 6.1 各国活跃度与图像密度时间序列对比
def plot_activity_img_time_series():
    plt.figure(figsize=(16, 12))
    
    for i, country in enumerate(selected_countries):
        country_data = df_combined[df_combined['country'] == country]
        
        plt.subplot(2, 3, i+1)
        # 绘制活跃度（人均推文数）
        plt.plot(country_data['date'], country_data['per_capita_tweet'], label='人均推文数', color='blue', alpha=0.7)
        # 绘制图像密度
        plt.plot(country_data['date'], country_data['img_perc'] * 100, label='图像密度(%)', color='red', alpha=0.7)
        
        plt.title(f'{country}活跃度与图像密度对比', fontsize=12)
        plt.xlabel('日期', fontsize=10)
        plt.ylabel('指标值', fontsize=10)
        plt.legend(fontsize=8)
        plt.grid(True, linestyle='--', alpha=0.3)
        plt.tight_layout()
    
    plt.savefig('activity_img_time_series.png', dpi=300)
    plt.show()

# 6.2 活跃度与图像可用性散点图（调小点大小）
def plot_activity_vs_img_availability():
    plt.figure(figsize=(14, 8))
    sns.scatterplot(
        x='per_capita_tweet', y='img_avail_perc', 
        hue='country', data=df_combined,
        palette='Set1', s=30,  # 调小散点大小
        alpha=0.8, edgecolor='white', linewidth=0.5
    )
    
    plt.title('活跃度与图像可用性关系', fontsize=16)
    plt.xlabel('人均推文数', fontsize=13)
    plt.ylabel('图像可用性', fontsize=13)
    plt.legend(title='国家', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('activity_vs_img_availability.png', dpi=300)
    plt.show()

# 6.3 活跃度与图像指标热力图（相关性）
def plot_correlation_heatmap():
    # 计算相关性矩阵
    corr_matrix = df_combined[['per_capita_tweet', 'img_perc', 'img_avail_perc']].corr()
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        corr_matrix, annot=True, fmt='.2f', cmap='YlOrRd', 
        vmin=-1, vmax=1, center=0,
        cbar_kws={'label': '相关系数'}
    )
    
    plt.title('活跃度与图像指标相关性热力图', fontsize=16)
    plt.tight_layout()
    plt.savefig('correlation_heatmap.png', dpi=300)
    plt.show()

# 6.4 重点国家活跃度与图像可用性异常检测（多国家对比、精简异常点）
def detect_anomalies_and_visualize():
    # 定义异常检测函数（Z-Score）
    def detect_anomaly(z_score, threshold=3):
        return abs(z_score) > threshold
    
    # 计算 Z-Score
    for col in ['per_capita_tweet', 'img_perc', 'img_avail_perc']:
        df_combined[f'{col}_zscore'] = (df_combined[col] - df_combined[col].mean()) / df_combined[col].std()
    
    # 标记异常
    df_combined['anomaly'] = df_combined.apply(
        lambda row: detect_anomaly(row['per_capita_tweet_zscore']) or 
                   detect_anomaly(row['img_perc_zscore']) or
                   detect_anomaly(row['img_avail_perc_zscore']),
        axis=1
    )
    
    # 可视化异常（多国家对比）
    plt.figure(figsize=(16, 10))
    for i, country in enumerate(selected_countries[:5]):  # 取前 5 个有研究价值的国家
        country_data = df_combined[df_combined['country'] == country]
        plt.subplot(2, 3, i + 1)
        
        plt.plot(country_data['date'], country_data['per_capita_tweet'], label='人均推文数', color='blue', alpha=0.7)
        plt.plot(country_data['date'], country_data['img_avail_perc'] * 100, label='图像可用性(%)', color='green', alpha=0.7)
        
        # 筛选最极端的异常点（Z-Score 最大的 3 个）
        anomaly_data = country_data[country_data['anomaly']]
        top_anomalies = anomaly_data.sort_values(by='per_capita_tweet_zscore', ascending=False).head(3)
        
        if not top_anomalies.empty:
            plt.scatter(top_anomalies['date'], top_anomalies['per_capita_tweet'], 
                        color='red', s=80, zorder=5, label='极端异常点')
        
        plt.title(f'{country}活跃度与图像可用性异常检测', fontsize=12)
        plt.xlabel('日期', fontsize=10)
        plt.ylabel('指标值', fontsize=10)
        plt.legend(fontsize=8)
        plt.grid(True, linestyle='--', alpha=0.3)
        plt.tight_layout()
    
    plt.savefig('multi_country_anomaly_detection.png', dpi=300)
    plt.show()

# 6.5 活跃度与图像可用性联合分布（3D 散点图，调整点大小和透明度）
def plot_3d_scatter():
    fig = px.scatter_3d(
        df_combined,
        x='per_capita_tweet', y='img_perc', z='img_avail_perc',
        color='country', symbol='country',
        hover_data=['date', 'num_tweet', 'num_user'],
        title='活跃度与图像指标3D散点图',
        size_max=24,  # 调小点大小到最小
        size='num_tweet',
        opacity=1  # 调整透明度
    )
    
    fig.update_layout(
        height=1000,
        scene=dict(
            xaxis=dict(title='人均推文数'),
            yaxis=dict(title='图像密度'),
            zaxis=dict(title='图像可用性')
        )
    )
    
    fig.show()

# 6.6 国家间活跃度与图像可用性对比（Pyecharts 折线图，Jupyter 中展示）
def plot_country_comparison():
    # 计算各国指标均值
    country_mean = df_combined.groupby('country').agg(
        avg_per_capita_tweet=('per_capita_tweet', 'mean'),
        avg_img_perc=('img_perc', 'mean'),
        avg_img_avail_perc=('img_avail_perc', 'mean')
    ).reset_index()
    
    # 准备 Pyecharts 数据
    x_data = country_mean['country'].tolist()
    
    line = (
        Line(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, width="1000px", height="600px"))
        .add_xaxis(xaxis_data=x_data)
        .add_yaxis(
            series_name="人均推文数",
            y_axis=country_mean['avg_per_capita_tweet'].tolist(),
            label_opts=opts.LabelOpts(is_show=True, position="top")
        )
        .add_yaxis(
            series_name="图像密度",
            y_axis=[x * 100 for x in country_mean['avg_img_perc'].tolist()],
            label_opts=opts.LabelOpts(is_show=True, position="top")
        )
        .add_yaxis(
            series_name="图像可用性",
            y_axis=[x * 100 for x in country_mean['avg_img_avail_perc'].tolist()],
            label_opts=opts.LabelOpts(is_show=True, position="top")
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="各国活跃度与图像指标均值对比"),
            toolbox_opts=opts.ToolboxOpts(is_show=True),
            xaxis_opts=opts.AxisOpts(name="国家"),
            yaxis_opts=opts.AxisOpts(name="指标值"),
            legend_opts=opts.LegendOpts(pos_top="5%")
        )
    )
    
    # 在 Jupyter 中展示 Pyecharts 图表
    display(HTML(line.render_embed()))

# 执行可视化
plot_activity_img_time_series()
plot_activity_vs_img_availability()
plot_correlation_heatmap()
detect_anomalies_and_visualize()
plot_3d_scatter()
plot_country_comparison()